In [1]:
import pytrends
import pandas as pd
import numpy as np
import datetime
import time
from pytrends.request import TrendReq

In [2]:
# connect to Google and set up time out to avoid reaching rate limit
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25))

Crashes with too many words, so run keywords in parts:
kw_list = [['cough', 'fever', 'chills'], 
    ['smell'], 
    ['fatigue', 'diarrhea', 'taste'], 
    ['lysol', 'face mask', 'covid stimulus check'], 
    ['breath', 'throat', 'vomit']]

In [5]:
# create empty dataframe to store the results
df = pd.DataFrame()
kw_list =  [['fatigue', 'diarrhea', 'taste']]
us_state_list = ['US-AL', 'US-AK', 'US-AZ', 'US-AR' , 'US-CA', 'US-CO', 'US-CT', 'US-DE',
                   'US-FL', 'US-GA', 'US-HI', 'US-ID', 'US-IL', 'US-IN', 'US-IA', 'US-KS', 'US-KY', 'US-LA',
                   'US-ME', 'US-MD', 'US-MA', 'US-MI', 'US-MN', 'US-MS', 'US-MO', 'US-MT', 'US-NE',
                   'US-NV', 'US-NH', 'US-NJ', 'US-NM', 'US-NY', 'US-NC', 'US-ND', 'US-OH', 'US-OK', 'US-OR',
                   'US-PA', 'US-RI', 'US-SC', 'US-SD', 'US-TN', 'US-TX', 'US-UT', 'US-VT', 'US-VA', 'US-WA',
                   'US-WV', 'US-WI', 'US-WY']


# decide the timeframe we wanted
# this is more flexible for use later since we can decide the time span we wanted
# though we do want to make sure the week definition will be the same as HHSdataset
time_frame = pd.DataFrame({
    'start': pd.date_range("2020-07-31", periods=25, freq="7D").astype('str').values,
    'end': pd.date_range("2020-08-06", periods=25, freq="7D").astype('str').values,
})
time_frame['combined'] = time_frame['start'] + ' ' + time_frame['end']


for keyword in range(len(kw_list)):
    for i in range(len(us_state_list)):
        for j in range(len(time_frame)):
            pytrends.build_payload(kw_list[keyword], cat=0, timeframe=time_frame['combined'][j], geo=us_state_list[i])
            fragment_df = pytrends.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=True)
            fragment_df['date'] = time_frame['start'][j]
            df = df.append(fragment_df)
            time.sleep(5)
            
        time.sleep(10)

    if keyword == 0:
        df = df.reset_index()
        all_df  = df.copy()
        df = pd.DataFrame()
        time.sleep(60)
        print("Requested: set", keyword)

    else:
        df = df.reset_index()
        all_df = pd.merge(all_df, df.drop("geoName", axis=1), left_on=['geoCode', 'date'], right_on=['geoCode', 'date'])
        df = pd.DataFrame()
            
        time.sleep(60)
        print("Requested: set", keyword)

ResponseError: The request failed: Google returned a response with code 429.

In [7]:
df.to_csv("cough_fever_chills.csv")

In [9]:
cough_fever_chills = pd.read_csv("cough_fever_chills.csv", index_col=0)
cough_fever_chills

,index,geoCode,cough,fever,chills,date
0,Birmingham AL,630,28.0,65.0,7.0,2020-07-31
1,Columbus GA,522,40.0,60.0,0.0,2020-07-31
2,Dothan AL,606,50.0,50.0,0.0,2020-07-31
3,Huntsville-Decatur (Florence) AL,691,20.0,74.0,6.0,2020-07-31
4,Meridian MS,711,0.0,0.0,0.0,2020-07-31
...,...,...,...,...,...,...
7520,Cheyenne WY-Scottsbluff NE,759,100.0,0.0,0.0,2021-01-15
7521,Denver CO,751,0.0,0.0,0.0,2021-01-15
7522,Idaho Falls-Pocatello ID,758,0.0,100.0,0.0,2021-01-15
7523,Rapid City SD,764,0.0,0.0,0.0,2021-01-15


In [12]:
lysol_facemask_stimulus=  pd.read_csv('lysol_facemask_stimulus.csv', index_col=0)
lysol_facemask_stimulus

,index,covid stimulus check,date,face mask,geoCode,lysol
0,Birmingham AL,0.0,2020-07-31,73.0,630,27.0
1,Columbus GA,0.0,2020-07-31,67.0,522,33.0
2,Dothan AL,0.0,2020-07-31,75.0,606,25.0
3,Huntsville-Decatur (Florence) AL,6.0,2020-07-31,81.0,691,13.0
4,Meridian MS,0.0,2020-07-31,100.0,711,0.0
...,...,...,...,...,...,...
7520,Cheyenne WY-Scottsbluff NE,0.0,2021-01-15,0.0,759,0.0
7521,Denver CO,0.0,2021-01-15,0.0,751,0.0
7522,Idaho Falls-Pocatello ID,0.0,2021-01-15,0.0,758,0.0
7523,Rapid City SD,0.0,2021-01-15,0.0,764,0.0


In [13]:
smell = pd.read_csv('smell.csv', index_col=0)
smell

,index,date,geoCode,smell
0,Birmingham AL,2020-07-31,630,52.0
1,Columbus GA,2020-07-31,522,48.0
2,Dothan AL,2020-07-31,606,58.0
3,Huntsville-Decatur (Florence) AL,2020-07-31,691,75.0
4,Meridian MS,2020-07-31,711,0.0
...,...,...,...,...
7520,Cheyenne WY-Scottsbluff NE,2021-01-15,759,100.0
7521,Denver CO,2021-01-15,751,0.0
7522,Idaho Falls-Pocatello ID,2021-01-15,758,0.0
7523,Rapid City SD,2021-01-15,764,0.0


In [18]:
data = pd.merge(smell, lysol_facemask_stimulus.drop('index', axis=1), left_on = ['geoCode', 'date'], right_on = ['geoCode', 'date'])
data = pd.merge(data, cough_fever_chills.drop('index', axis=1), left_on = ['geoCode', 'date'], right_on = ['geoCode', 'date'])
data.to_csv('7_kewords.csv')